# MongoDB

## Conexión con MongoDB y borrado de datos

In [ ]:
import pymongo 
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import json

from pprintpp import pprint as pp

In [ ]:
# connect to database
connection = MongoClient('localhost', 27017)

In [ ]:
# Borramos la base de datos
connection.drop_database('black')

# Pymongo permite una sintaxis practicamente igual que la original
db = connection.black

## Lectura en Pandas

Leemos los datos desde el fichero Excel y los desnormalizamos

In [ ]:
import pandas as pd
import datetime

In [ ]:
df_mov = pd.read_excel("../../data/black.xlsx", sheet_name= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheet_name= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

## Carga en MongoDB

Gracias a Pandas convertimos el dataset en formato JSON, que es el que vamos a insertar en MongoDB prácticamente sin cambios.

In [ ]:
json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

In [ ]:
pp(json_list[0])

En MongoDB vamos a crear una única colección donde vamos a guardar todos los datos desnormalizados:

<img src="images/Modelo%20MongoDB.png" width=300 height=100>

<br><br> 

En la inserción de la información en mongo realizan algunas tareas de limpieza:

- Eliminar los campos nulos (no todos)
- Convertir la fecha a un formato DateTime, de forma que mongo inserte el dato correctamente
- Eliminar campos que no queremos tener en el modelo (id_miembro)

In [ ]:
for movimiento_json in json_list:
    movimiento_json['fecha'] = datetime.datetime.fromtimestamp(movimiento_json['fecha'] / 1e3)
    
    del movimiento_json['id_miembro']

    if  movimiento_json['comercio'] == None:
        del movimiento_json['comercio']
    
    db.movimientos.insert_one(movimiento_json)

## Querys

Todas las querys de este caso de uso se pueden resolver directamente por la base de datos ...

### Los 10 movimientos mas caros

In [ ]:
rs = db.movimientos.find({}, {
    "nombre": 1,
    "fecha": 1,
    "actividad_completa": 1,
    "importe": 1,
    "_id" : 0
}).limit(10).sort([("importe", -1)])

In [ ]:
pd.DataFrame(list(rs))

### Los 10 movimientos mas caros por actividad

In [ ]:
rs = db.movimientos.find({
    "actividad": "HOGAR"
}, {
    "nombre": 1,
    "fecha": 1,
    "actividad_completa": 1,
    "importe": 1,
    "_id" : 0
}).limit(10).sort([("importe", -1)])

In [ ]:
pd.DataFrame(list(rs))

### Las 10 personas que mas han gastado

En este caso, al obtener un dato agregado, necesitamos utilizar la función **aggregate()** de mongo

In [ ]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$nombre", "importe" : { "$sum" : "$importe"}}},
        {"$sort" : {"importe" : -1}},
        {"$limit" : 10},
        {"$project" : {"_id" : 0, "nombre": "$_id", "importe": 1}}
    ])

In [ ]:
df = pd.DataFrame(list(rs))
df

### ¿Que comercio es más popular?

Este es un caso especial de agregación, ya que cuenta los distintos. Dentro del flujo de la función se realizan varios pasos para resolver esta query:

- Formar un conjunto con los distintas personas que han comprado en cada comercio
- Desagregar el conjunto
- Agrupar los comercios, contanto los distintos elementos de cada conjunto

In [ ]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$comercio", "nombres" : { "$addToSet" : "$nombre"}}},
        {"$unwind" : "$nombres"},
        {"$group" : { "_id" : "$_id", "cuenta" : { "$sum" : 1}}},
        {"$sort" : {"cuenta" : -1}},
        {"$project" : {"_id" : 0, "comercio": "$_id", "cuenta": 1}},
        {"$match": {"comercio": { "$exists": True, "$ne": None }}},
        {"$limit" : 10}
    ])

In [ ]:
df = pd.DataFrame(list(rs))
df

Truco ...

En este tipo de querys complicadas podemos ver los resultados parciales ...

In [ ]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$comercio", "nombres" : { "$addToSet" : "$nombre"}}},
        {"$unwind" : "$nombres"}
    ])

pd.DataFrame(list(rs)).head()